In [27]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [28]:
trainimage_tensor = torch.load("./imagepreprocessdataset/trainimagetensor.pt")
validimage_tensor = torch.load("./imagepreprocessdataset/validimagetensor.pt")
trainlabel = torch.load("./imagepreprocessdataset/trainlabel.pt")
validlabel = torch.load("./imagepreprocessdataset/validlabel.pt")

In [34]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class CustomDataset(Dataset): 
  def __init__(self, x, y, transforms):
    self.x_data = x
    self.y_data = y
    self.transforms = transforms

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.FloatTensor(self.x_data[idx])
    x = self.transforms(x)
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

In [32]:
train_transform = transforms.Compose([
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])
valid_transform = transforms.Compose([
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])

In [31]:
trainlabel = trainlabel.unsqueeze(-1)
validlabel = validlabel.unsqueeze(-1)

In [35]:
train_dataset = CustomDataset(trainimage_tensor, trainlabel, transforms = train_transform)
trainloader = DataLoader(train_dataset, batch_size=32, num_workers=0)
valid_dataset = CustomDataset(validimage_tensor, validlabel, transforms = valid_transform)
validloader = DataLoader(valid_dataset, batch_size=32, num_workers=0)

In [36]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)

    # Loss Function
    criterion = nn.MSELoss().to(device)
    best_mae = 9999
    
    for epoch in range(1,11):
        print("epoch :",epoch)
        model.train()
        train_loss = []
        #meta_loader = [i for i in enumerate(train_meta_loader)]
        for idx, data in enumerate(train_loader):
            print(idx)
            #img, label = data[0], data[1]
            img, label = data[0], data[1]
            print(img[0])
            print(label[0])
            # meta = train_meta_loader[idx]
            #meta = meta_loader[idx][1]
            img, label = img.float().to(device), label.float().to(device)
            #meta = meta.float().to(device)
            optimizer.zero_grad()

            # Data -> Model -> Output
            print("model start")
            logit = model(img)
            #print(logit)
            print("model end")
            # Calc loss
            loss = criterion(logit.squeeze(1), label)

            # backpropagation
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            
        if scheduler is not None:
            scheduler.step()
            
        # Evaluation Validation set
        vali_mae = validation(model, vali_loader, criterion, device)
        
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')
        
        # Model Saved
        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), './saved/best_model.pth')
            print('Model Saved.')

In [17]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    with torch.no_grad():
        #meta_loader = [i for i in enumerate(val_meta_loader)]
        for idx, data in enumerate(vali_loader):
            img, label = data[0], data[1]
            # meta = val_meta_loader[idx]
            #meta = meta_loader[idx][1]
            img, label = img.float().to(device), label.float().to(device)
            #meta = meta.float().to(device)
            
            logit = model(img)
            loss = criterion(logit.squeeze(1), label)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss  

In [18]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, 1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model

In [37]:
#from resnet import
import torch.optim as optim
#model = resnet18().to(device)
model = ResNet(BasicBlock, [2, 2, 2, 2])
optimizer = optim.SGD(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [12]:
import torchsummary
torchsummary.summary(model, (3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
       BasicBlock-11           [-1, 64, 32, 32]               0
           Conv2d-12           [-1, 64, 32, 32]          36,864
      BatchNorm2d-13           [-1, 64, 32, 32]             128
             ReLU-14           [-1, 64,

In [38]:
train(model, optimizer, trainloader, validloader, scheduler, device)

epoch : 1
0
model start
tensor([[ 0.0596],
        [ 0.0212],
        [-0.0012],
        [ 0.1746],
        [ 0.6114],
        [ 0.0785],
        [ 0.0765],
        [ 0.6815],
        [ 0.1325],
        [ 0.0963],
        [ 0.6294],
        [ 0.2077],
        [ 0.1137],
        [ 0.3903],
        [ 0.1296],
        [ 0.3068],
        [ 0.5364],
        [ 0.0666],
        [ 0.2248],
        [ 0.1764],
        [ 0.0605],
        [ 0.2353],
        [ 0.2121],
        [ 0.0943],
        [ 0.2015],
        [ 0.6698],
        [ 0.0458],
        [ 0.1434],
        [ 0.0954],
        [ 0.0367],
        [ 0.1021],
        [ 0.1923]], grad_fn=<AddmmBackward>)
model end


C:\ProgramData\Anaconda3\envs\wonyong\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


1
model start
tensor([[ 83.7631],
        [ 92.6958],
        [355.4938],
        [ 85.8503],
        [ 91.9314],
        [ 92.6648],
        [ 86.5690],
        [ 95.2026],
        [ 93.4666],
        [ 89.5297],
        [ 96.1613],
        [ 89.7331],
        [ 90.7674],
        [ 99.5398],
        [105.3996],
        [ 85.0789],
        [102.6573],
        [101.4536],
        [ 93.3485],
        [102.4121],
        [ 97.8105],
        [ 95.2021],
        [106.2305],
        [110.7377],
        [ 88.2336],
        [110.5712],
        [146.2282],
        [ 85.0286],
        [119.6905],
        [106.6086],
        [ 88.3104],
        [124.5537]], grad_fn=<AddmmBackward>)
model end
2
model start
tensor([[20.1247],
        [20.0821],
        [20.1308],
        [20.0225],
        [20.0650],
        [19.9803],
        [16.9681],
        [19.5177],
        [19.9323],
        [18.5848],
        [20.5636],
        [19.3572],
        [18.7053],
        [20.7004],
        [18.4659],
        [18

KeyboardInterrupt: 

In [25]:
for idx, data in enumerate(trainloader):
    print(data[0].shape)
    break

torch.Size([32, 3, 128, 128])
